In [ ]:
#from google.colab import drive
#drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install pyspark
!pip install openclean
!pip install uszipcode
!pip install humanfriendly

In [ ]:
import pandas as pd
import uszipcode
import gzip
import humanfriendly
import os
import numpy as np

In [ ]:
from openclean.data.source.socrata import Socrata
from openclean.profiling.column import DefaultColumnProfiler
from openclean.profiling.anomalies.sklearn import DBSCANOutliers
from openclean.cluster.key import KeyCollision
from openclean.function.value.key.fingerprint import Fingerprint

In [ ]:
!pip install datatable

In [ ]:
import datatable as dt
pd_data = dt.fread("../data/311_Service_Requests_from_2010_to_Present.csv")

In [ ]:
pd_data = pd.read_csv("../data/311_Service_Requests_from_2010_to_Present.csv")

In [ ]:
pd_data.head()

,Unnamed: 0,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Description,Resolution Action Updated Date,Community Board,BBL,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Open Data Channel Type,Park Facility Name,Park Borough,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,0,51560911,08/17/2021 07:34:45 AM,08/17/2021 09:34:29 AM,NYPD,New York City Police Department,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,"11,421",92-13 92 AVENUE,92 AVENUE,92 STREET,WOODHAVEN BOULEVARD,92 STREET,WOODHAVEN BOULEVARD,NaN,WOODHAVEN,92 AVENUE,NaN,Closed,NaN,The Police Department issued a summons in resp...,08/17/2021 09:34:31 AM,09 QUEENS,"4,089,860,017",QUEENS,"1,025,640","190,277",MOBILE,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41,-74,"(40.68884640295723, -73.85075384808735)"
1,1,51560923,08/17/2021 06:04:48 PM,08/17/2021 07:25:36 PM,NYPD,New York City Police Department,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,"10,468",CLAFLIN AVENUE,CLAFLIN AVENUE,CLAFLIN AVENUE,WEST 195 STREET,CLAFLIN AVENUE,WEST 195 STREET,NaN,NaN,NaN,NaN,Closed,NaN,The Police Department responded and upon arriv...,08/17/2021 07:25:44 PM,08 BRONX,nan,BRONX,"1,011,740","256,343",PHONE,Unspecified,BRONX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41,-74,"(40.87023400587434, -73.90060480716431)"
2,2,50469886,05/04/2021 11:40:00 AM,07/21/2021 01:55:00 PM,DOT,Department of Transportation,Street Light Condition,Street Light Out,NaN,"11,214",NaN,NaN,NaN,NaN,NEW UTRECHT AVENUE,84 STREET,INTERSECTION,BROOKLYN,NaN,NaN,Closed,NaN,Service Request status for this request is ava...,07/21/2021 01:55:00 PM,11 BROOKLYN,nan,BROOKLYN,"983,735","161,202",UNKNOWN,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41,-74,"(40.60913860285155, -74.00185479490075)"
3,3,50470191,05/04/2021 04:31:32 PM,NaN,DPR,Department of Parks and Recreation,New Tree Request,For One Address,Street,"11,229",2041 EAST 18 STREET,EAST 18 STREET,AVENUE T,AVENUE U,AVENUE T,AVENUE U,NaN,BROOKLYN,EAST 18 STREET,NaN,In Progress,NaN,NaN,NaN,15 BROOKLYN,"3,073,230,065",BROOKLYN,"997,085","158,030",UNKNOWN,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41,-74,"(40.600422830424975, -73.95378022131071)"
4,4,50470218,05/05/2021 01:47:05 AM,10/01/2021 10:53:17 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Signs of Rodents,1-2 Family Dwelling,"11,420",115-44 135 STREET,135 STREET,115 AVENUE,116 AVENUE,115 AVENUE,116 AVENUE,ADDRESS,South Ozone Park,135 STREET,NaN,Closed,NaN,This service request was closed because the De...,05/05/2021 01:47:05 AM,10 QUEENS,"4,116,770,025",QUEENS,"1,038,085","187,070",MOBILE,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41,-74,"(40.67997699346374, -73.8059047916443)"


In [ ]:
pd_data = pd_data.rename(columns={"Created Date":"Created_Date",
                                  "Closed Date":"Closed_Date",
                                  "Agency Name":"Agency_Name",
                                  "Complaint Type":"Complaint_Type",
                                  "Location Type":"Location_Type",
                                  "Incident Zip":"Incident_Zip",
                                  "Incident Address":"Incident_Address",
                                  "Street Name":"Street_Name",
                                  "Cross Street 1":"Cross_Street_1",
                                  "Cross Street 2":"Cross_Street_2",
                                  "Intersection Street 1":"Intersection_Street_1",
                                  "Intersection Street 2":"Intersection_Street_2",
                                  "Address Type":"Address_Type",
                                  "Facility Type":"Facility_Type",
                                  "Due Date":"Due_Date",
                                  "Resolution Description":"Resolution_Description",
                                  "Resolution Action Updated Date":"Resolution_Action_Updated_Date",
                                  "Community Board":"Community_Board",
                                  "Park Borough":"Park_Borough"})

### ZIP Code mapping

Loading the Reference Data for zip codes.

In [ ]:
zip_data = pd.read_csv("../data/nyc-zip-codes.csv")

In [ ]:
zip_data.head()

,Borough,Neighborhood,ZipCode
0,Bronx,Central Bronx,10453
1,Bronx,Central Bronx,10457
2,Bronx,Central Bronx,10460
3,Bronx,Bronx Park and Fordham,10458
4,Bronx,Bronx Park and Fordham,10467


In [ ]:
len(zip_data.ZipCode.unique())

178

Unique Zip codes found in our dataset are way greater than the unique codes found in the reference data

In [ ]:
len(pd_data.Incident_Zip.unique())

1378

Creating a function to map valid zip codes using reference data and flagging other values as unspecified.

In [ ]:
def check_zip(z):
    
  if z in zip_data.ZipCode.unique():
        return int(z) 
  
  else:
        return 'Unspecified'

In [ ]:
pd_data.Incident_Zip = pd_data.Incident_Zip.map(lambda x: check_zip(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
len(pd_data.Incident_Zip.unique())

179

In [ ]:
pd_data.head()

,Unnamed: 0,Unique Key,Created_Date,Closed_Date,Agency,Agency_Name,Complaint_Type,Descriptor,Location_Type,Incident_Zip,Incident_Address,Street_Name,Cross_Street_1,Cross_Street_2,Intersection_Street_1,Intersection_Street_2,Address_Type,City,Landmark,Facility_Type,Status,Due_Date,Resolution_Description,Resolution_Action_Updated_Date,Community_Board,BBL,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Open Data Channel Type,Park Facility Name,Park_Borough,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,0,51560911,08/17/2021 07:34:45 AM,08/17/2021 09:34:29 AM,NYPD,New York City Police Department,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,11421,92-13 92 AVENUE,92 AVENUE,92 STREET,WOODHAVEN BOULEVARD,92 STREET,WOODHAVEN BOULEVARD,NaN,WOODHAVEN,92 AVENUE,NaN,Closed,NaN,The Police Department issued a summons in resp...,08/17/2021 09:34:31 AM,09 QUEENS,"4,089,860,017",QUEENS,"1,025,640","190,277",MOBILE,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41,-74,"(40.68884640295723, -73.85075384808735)"
1,1,51560923,08/17/2021 06:04:48 PM,08/17/2021 07:25:36 PM,NYPD,New York City Police Department,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,10468,CLAFLIN AVENUE,CLAFLIN AVENUE,CLAFLIN AVENUE,WEST 195 STREET,CLAFLIN AVENUE,WEST 195 STREET,NaN,NaN,NaN,NaN,Closed,NaN,The Police Department responded and upon arriv...,08/17/2021 07:25:44 PM,08 BRONX,nan,BRONX,"1,011,740","256,343",PHONE,Unspecified,BRONX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41,-74,"(40.87023400587434, -73.90060480716431)"
2,2,50469886,05/04/2021 11:40:00 AM,07/21/2021 01:55:00 PM,DOT,Department of Transportation,Street Light Condition,Street Light Out,NaN,11214,NaN,NaN,NaN,NaN,NEW UTRECHT AVENUE,84 STREET,INTERSECTION,BROOKLYN,NaN,NaN,Closed,NaN,Service Request status for this request is ava...,07/21/2021 01:55:00 PM,11 BROOKLYN,nan,BROOKLYN,"983,735","161,202",UNKNOWN,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41,-74,"(40.60913860285155, -74.00185479490075)"
3,3,50470191,05/04/2021 04:31:32 PM,NaN,DPR,Department of Parks and Recreation,New Tree Request,For One Address,Street,11229,2041 EAST 18 STREET,EAST 18 STREET,AVENUE T,AVENUE U,AVENUE T,AVENUE U,NaN,BROOKLYN,EAST 18 STREET,NaN,In Progress,NaN,NaN,NaN,15 BROOKLYN,"3,073,230,065",BROOKLYN,"997,085","158,030",UNKNOWN,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41,-74,"(40.600422830424975, -73.95378022131071)"
4,4,50470218,05/05/2021 01:47:05 AM,10/01/2021 10:53:17 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Signs of Rodents,1-2 Family Dwelling,11420,115-44 135 STREET,135 STREET,115 AVENUE,116 AVENUE,115 AVENUE,116 AVENUE,ADDRESS,South Ozone Park,135 STREET,NaN,Closed,NaN,This service request was closed because the De...,05/05/2021 01:47:05 AM,10 QUEENS,"4,116,770,025",QUEENS,"1,038,085","187,070",MOBILE,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41,-74,"(40.67997699346374, -73.8059047916443)"


### Community Board

Reference data for Community Board is found here: https://communityprofiles.planning.nyc.gov/

In [ ]:
community_data = pd.read_csv("../data/CommunityDistrictData.csv")

In [ ]:
community_data.head()

,the_geom,cartodb_id,the_geom_webmercator,acres,acs_tooltip,acs_tooltip_2,acs_tooltip_3,area_sqmi,borocd,cb_email,cb_website,cd_full_title,cd_short_title,cd_son_fy2018,cd_tot_bldgs,cd_tot_resunits,count_hosp_clinic,count_libraries,count_parks,count_public_schools,crime_count,crime_count_boro,crime_count_nyc,crime_per_1000,crime_per_1000_boro,crime_per_1000_nyc,female_10_14,female_15_19,female_20_24,female_25_29,female_30_34,female_35_39,female_40_44,female_45_49,female_5_9,female_50_54,female_55_59,female_60_64,female_65_69,female_70_74,...,pct_asian_nh,pct_bach_deg,pct_bach_deg_boro,pct_bach_deg_nyc,pct_black_nh,pct_clean_strts,pct_clean_strts_boro,pct_clean_strts_nyc,pct_foreign_born,pct_hh_rent_burd,pct_hh_rent_burd_boro,pct_hh_rent_burd_nyc,pct_hispanic,pct_other_nh,pct_served_parks,pct_white_nh,pop_2000,pop_2010,pop_acs,pop_change_00_10,poverty_rate,poverty_rate_boro,poverty_rate_nyc,puma,shared_puma,shared_puma_cd,son_issue_1,son_issue_2,son_issue_3,total_lot_area,under18_rate,under18_rate_boro,under18_rate_nyc,unemployment_boro,unemployment,unemployment_nyc,v_pluto,v_acs,v_facdb,v_crime
0,NaN,1,NaN,976.3,American Community Survey 2014-2018 5-Year Est...,American Community Survey (ACS) 2013-2017 5-ye...,2010 Census population counts for floodplain a...,1.5,101,man01@cb.nyc.gov,www1.nyc.gov/site/manhattancb1/index.page,Manhattan Community District 1,Manhattan CD 1,NaN,1725,45629,10,2,13,19,1055,26271,92480,6.8,16.1,11,1.1,2.4,3.9,7.2,6.5,4.7,3.6,2.9,1.8,2.3,2.4,2.6,2.4,1.8,...,15.5,82.2,60.8,37.4,2.2,95.4,94.1,95.1,23.3,30.1,36.4,44.2,6.5,3.6,100,72.2,34420,60978,154636,0.77,8.8,14.4,19.8,3810,True,Manhattan CD 2,Infrastructure resiliency,Traffic,Other,2881166.373,12.7,14.4,20.9,3.8,2.8,4.4,20v4,Y2014-2018,6/24/20,2019
1,NaN,2,NaN,865.9,American Community Survey 2014-2018 5-Year Est...,American Community Survey (ACS) 2013-2017 5-ye...,2010 Census population counts for floodplain a...,1.4,102,bgormley@cb.nyc.gov,www.nyc.gov/html/mancb2,Manhattan Community District 2,Manhattan CD 2,NaN,5256,60610,19,3,6,8,2313,26271,92480,15.0,16.1,11,1.1,2.4,3.9,7.2,6.5,4.7,3.6,2.9,1.8,2.3,2.4,2.6,2.4,1.8,...,15.5,82.2,60.8,37.4,2.2,96.3,94.1,95.1,23.3,30.1,36.4,44.2,6.5,3.6,100,72.2,93119,90016,154636,-0.03,8.8,14.4,19.8,3810,True,Manhattan CD 1,Parks,Schools,Senior services,2231472.120,12.7,14.4,20.9,3.8,2.8,4.4,20v4,Y2014-2018,6/24/20,2019
2,NaN,3,NaN,1076.9,American Community Survey 2014-2018 5-Year Est...,American Community Survey (ACS) 2013-2017 5-ye...,2010 Census population counts for floodplain a...,1.7,103,mn03@cb.nyc.gov,www.nyc.gov/html/mancb3,Manhattan Community District 3,Manhattan CD 3,NaN,4644,82557,41,5,13,42,2360,26271,92480,15.2,16.1,11,1.5,2.9,4.2,6.4,4.1,3.2,3.0,3.2,1.2,2.8,3.3,3.4,2.9,2.1,...,30.6,43.3,60.8,37.4,8.4,91.0,94.1,95.1,34.5,38.4,36.4,44.2,25.1,2.6,100,33.3,164407,163277,154995,-0.01,19.3,14.4,19.8,3809,False,NaN,Affordable housing,Senior services,Homelessness,2985434.526,11.4,14.4,20.9,3.8,3.6,4.4,20v4,Y2014-2018,6/24/20,2019
3,NaN,4,NaN,1131.6,American Community Survey 2014-2018 5-Year Est...,American Community Survey (ACS) 2013-2017 5-ye...,2010 Census population counts for floodplain a...,1.8,104,jbodine@cb.nyc.gov,www.nyc.gov/mcb4,Manhattan Community District 4,Manhattan CD 4,NaN,3829,85706,38,2,7,29,2792,26271,92480,18.1,16.1,11,0.9,1.7,4.0,7.2,6.4,3.8,3.0,2.9,1.0,2.3,2.7,2.7,2.1,2.1,...,17.2,74.1,60.8,37.4,5.2,95.6,94.1,95.1,30.8,33.9,36.4,44.2,15.5,3.0,94,59.1,87479,103245,154496,0.18,11.3,14.4,19.8,3807,True,Manhattan CD 5,Affordable housing,"Land use trends (zoning, development, neighbor...",Traffic,3110433.364,8.2,14.4,20.9,3.8,3.5,4.4,20v4,Y2014-2018,6/24/20,2019
4,NaN,5,NaN,1005.4,American Community Survey 2014-2018 5-Year Est...,American Community Survey (ACS) 2013-2017 5-ye...,2010 Census population counts for floodplain a...,1.6,105,office@cb5.org,www.cb5.org,Manhattan Community District 5,Manhattan CD 5,NaN,3127,49436,23,7,3,13,4538,26271,92480,29.4,16.1,11,0.9,1.7,4.0,7.2,6.4,3.8,3.0,2.9,1

Checking for unique values of Community Boards in the reference data and our dataset.

In [ ]:
len(community_data.cd_short_title.unique())

59

In [ ]:
len(pd_data.Community_Board.unique())

80

Checking the format of names.

In [ ]:
community_data.cd_short_title.head(10)

0     Manhattan CD 1
1     Manhattan CD 2
2     Manhattan CD 3
3     Manhattan CD 4
4     Manhattan CD 5
5     Manhattan CD 6
6     Manhattan CD 7
7     Manhattan CD 8
8     Manhattan CD 9
9    Manhattan CD 10
Name: cd_short_title, dtype: object

In [ ]:
pd_data.Community_Board.head(10)

0        09 QUEENS
1         08 BRONX
2      11 BROOKLYN
3      15 BROOKLYN
4        10 QUEENS
5        10 QUEENS
6         11 BRONX
7         04 BRONX
8      06 BROOKLYN
9    0 Unspecified
Name: Community_Board, dtype: object

As seen above, there seems to be difference in formatting of the values in both datasets. We will make small changes to the reference data and our dataset so that the Community titles in reference data matches with the Community Board column in our dataset.

In [ ]:
pd.options.mode.chained_assignment = None 

for row in community_data.index:
    upper = community_data.at[row, "cd_short_title"].upper().replace(' CD','')
    num_first = "{}".format(upper[-2:] + ' ' + upper[:-2]).strip()
    community_data.cd_short_title.loc[row] = num_first

In [ ]:
community_data.cd_short_title.head(10)

0     1 MANHATTAN
1     2 MANHATTAN
2     3 MANHATTAN
3     4 MANHATTAN
4     5 MANHATTAN
5     6 MANHATTAN
6     7 MANHATTAN
7     8 MANHATTAN
8     9 MANHATTAN
9    10 MANHATTAN
Name: cd_short_title, dtype: object

In [ ]:
def community_changes(cb_string): 
  
  if cb_string.startswith('0'):
        return cb_string[1:]
  else:
        return cb_string

In [ ]:
pd_data.Community_Board = [community_changes(str(i)) for i in pd_data.Community_Board]

In [ ]:
pd_data.Community_Board.head(10)

0        9 QUEENS
1         8 BRONX
2     11 BROOKLYN
3     15 BROOKLYN
4       10 QUEENS
5       10 QUEENS
6        11 BRONX
7         4 BRONX
8      6 BROOKLYN
9     Unspecified
Name: Community_Board, dtype: object

Mapping Community Board names with the reference data and flagging other values as unspecified.

In [ ]:
def check_community(z):
    
  if z in community_data.cd_short_title.unique():
        return z 
  
  else:
        return 'Unspecified'

In [ ]:
pd_data.Community_Board = pd_data.Community_Board.map(lambda x: check_community(x))

In [ ]:
len(pd_data.Community_Board.unique())

60